### meteo con el pato

In [ ]:
import duckdb

%load_ext sql
conn = duckdb.connect() #duckdb.connect('/home/jcaubin/duck_test.db') #conexion a bbdd en memoria o en fichero
%sql conn --alias duckdb
%sql -l

In [2]:
#limite de filas a mostrar
%config SqlMagic.displaylimit = 100 

In [ ]:
%%sql 

create table meteo_today as
select * from read_csv('https://datos.madrid.es/egob/catalogo/300392-11041819-meteorologia-tiempo-real.csv');

In [ ]:
%%sql

create table estaciones as
select * from read_csv('https://datos.madrid.es/egob/catalogo/300360-1-meteorologicos-estaciones.csv')

In [ ]:
%%sql

create table magnitudes as
select * from read_csv('../maestras/parametros_meteo.csv')

In [ ]:
%sql select * from estaciones

In [ ]:
%%sql
describe meteo_today

In [ ]:
%%sql

unpivot meteo_today
on
H01, 
H02, 
H03, 
H04, 
H05, 
H06, 
H07, 
H08, 
H09, 
H10, 
H11, 
H12, 
H13, 
H14, 
H15, 
H16, 
H17, 
H18, 
H19, 
H20, 
H21, 
H22, 
H23, 
H24, 
into name h value v
;

In [ ]:
%%sql

create or replace table td as
select PROVINCIA, MUNICIPIO, ESTACION, MAGNITUD, PUNTO_MUESTREO,ANO, MES, DIA, cast(right(HORA,2) as int) H, VALOR	 from (
from meteo_today
unpivot (
    (VALOR)
for HORA in (
    H01, 
    H02, 
    H03, 
    H04, 
    H05, 
    H06, 
    H07, 
    H08, 
    H09, 
    H10, 
    H11, 
    H12, 
    H13, 
    H14, 
    H15, 
    H16, 
    H17, 
    H18, 
    H19, 
    H20, 
    H21, 
    H22, 
    H23, 
    H24 
    ) 
)
);

In [ ]:
%%sql 

create or replace  table tv as
select PROVINCIA, MUNICIPIO, ESTACION, MAGNITUD, PUNTO_MUESTREO,ANO, MES, DIA, cast(right(HORA,2) as int) H, VALIDEZ	 
from (
from meteo_today
unpivot (
    (VALIDEZ)
for HORA in (
    V01, 
    V02, 
    V03, 
    V04, 
    V05, 
    V06, 
    V07, 
    V08, 
    V09, 
    V10, 
    V11, 
    V12, 
    V13, 
    V14, 
    V15, 
    V16, 
    V17, 
    V18, 
    V19, 
    V20, 
    V21, 
    V22, 
    V23, 
    V24, 
    ) 
 ))
 ;


In [ ]:
%%sql

create or replace table meteo_madrid24 as
select td.*, tv.VALIDEZ, e.ESTACION ESTACION_DESC, e.ALTITUD
from td 
inner join tv on td.PROVINCIA=tv.PROVINCIA and 	td.MUNICIPIO=tv.MUNICIPIO and td.ESTACION=tv.ESTACION and 	td.MAGNITUD = tv.MAGNITUD
    and 	td.PUNTO_MUESTREO = tv.PUNTO_MUESTREO and td.ANO= tv.ANO and td.MES = tv.MES and TD.DIA = tv.DIA and td.H=tv.H
inner join estaciones e on td.estacion = e.CÓDIGO_CORTO    

In [ ]:
%%sql

select *
from meteo_madrid24
where
magnitud = 83 --temperatura
and ESTACION = 58 --el pardo
order by ano, mes, dia, h

In [ ]:
relation = conn.sql("""select *
from meteo_madrid24
where
magnitud = 83 --temperatura
and ESTACION = 58 --el pardo
 and VALIDEZ = 'V' --validos                   
order by ano, mes, dia, h""")
relation

In [ ]:
import hvplot.duckdb

relation.hvplot.line(y='VALOR', x='H')

In [ ]:
%%sql temp_data<<
PIVOT(
select estacion, h, valor
from meteo_madrid24
where
magnitud = 83 --temperatura
and validez = 'V'
)
on ESTACION
USING sum(VALOR)
group by h

In [ ]:
relation = conn.sql("""PIVOT(
select estacion_desc, h, valor
from meteo_madrid24
where
magnitud = 83 --temperatura
and validez = 'V'
)
on ESTACION_DESC
USING sum(VALOR)
group by h""")
relation

In [ ]:
relation.hvplot.line(y=['El Pardo', 'Escuelas Aguirre', 'Casa de Campo', 'Plaza España'], x='h')